### Customer Lifetime Value prediction

In [805]:
import pandas as pd

In [806]:
df = pd.read_csv(filepath_or_buffer="./data/customer_segmentation.csv",
                 encoding="cp1252")  # Specify the encoding!
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


#### Convert the date field

In [807]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

#### Create a new field / feature from existing ones

In [808]:
df['InvoiceYearMonth'] = df['InvoiceDate'].map(lambda date: 100 * date.year + date.month)
df['Revenue'] = df['UnitPrice'] * df['Quantity']  # This should be predicted!

In [809]:
df.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceYearMonth,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,201012,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,201012,20.34


In [810]:
df.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID,InvoiceYearMonth,Revenue
count,541909.000000,541909,541909.000000,406829.000000,541909.000000,541909.000000
mean,9.552250,2011-07-04 13:34:57.156386048,4.611114,15287.690570,201099.713989,17.987795
min,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000,201012.000000,-168469.600000
25%,1.000000,2011-03-28 11:34:00,1.250000,13953.000000,201103.000000,3.400000
50%,3.000000,2011-07-19 17:17:00,2.080000,15152.000000,201107.000000,9.750000
75%,10.000000,2011-10-19 11:27:00,4.130000,16791.000000,201110.000000,17.400000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000,201112.000000,168469.600000
std,218.081158,NaN,96.759853,1713.600303,25.788703,378.810824


In [811]:
filtered = df.drop(columns=['StockCode', 'Description'])
filtered = filtered.dropna()

# keep only rows that make sens, i.e. are either definitely costs or definitely profits
filtered = filtered[
    (filtered['Revenue'] > 0) & (filtered['Quantity'] > 0) | (filtered['Revenue'] < 0) & (
                filtered['Quantity'] < 0)]

max_date = df['InvoiceDate'].max()
date_limit = max_date - pd.Timedelta(days=180)
filtered = filtered[filtered['InvoiceDate'] >= date_limit]
# filtered = filtered['CustomerID'].apply(lambda x: int(x) if x == x else "")


positive_revenue = filtered[filtered['Revenue'] >= 0]
positive_revenue = positive_revenue.drop_duplicates(subset=['InvoiceNo'], keep='first')

invoice_counts = positive_revenue['CustomerID'].value_counts()

customers_with_multiple_invoices = invoice_counts[invoice_counts >= 2.0].keys()

print(len(customers_with_multiple_invoices))
filtered_aux = filtered[filtered['CustomerID'].isin(customers_with_multiple_invoices)]

print(filtered_aux.head(5))
filtered_aux.shape

2015
       InvoiceNo  Quantity         InvoiceDate  UnitPrice  CustomerID  \
223297    556479         3 2011-06-12 12:50:00       8.95     16983.0   
223298    556479         1 2011-06-12 12:50:00       1.65     16983.0   
223299    556479         1 2011-06-12 12:50:00       2.95     16983.0   
223300    556479         2 2011-06-12 12:50:00       2.95     16983.0   
223301    556479         6 2011-06-12 12:50:00       3.75     16983.0   

               Country  InvoiceYearMonth  Revenue  
223297  United Kingdom            201106    26.85  
223298  United Kingdom            201106     1.65  
223299  United Kingdom            201106     2.95  
223300  United Kingdom            201106     5.90  
223301  United Kingdom            201106    22.50  


(210890, 8)

In [812]:
rfmt = filtered_aux.groupby('CustomerID').agg(
    {'InvoiceDate': lambda date: (max_date - date.max()).days,
     'InvoiceNo': lambda num: len(num),
     'Revenue': lambda price: price.sum()},)


print(rfmt.loc[(rfmt['Revenue'].idxmax())])

rfmt.head(5)
rfmt.describe()

InvoiceDate         0.00
InvoiceNo         330.00
Revenue        164467.07
Name: 18102.0, dtype: float64


,InvoiceDate,InvoiceNo,Revenue
count,2015.000000,2015.000000,2015.000000
mean,29.451613,104.660050,2193.264780
std,32.209199,231.182819,7489.791621
min,0.000000,2.000000,-229.200000
25%,7.000000,33.000000,567.690000
50%,18.000000,62.000000,1013.260000
75%,40.000000,118.000000,1854.415000
max,171.000000,5621.000000,164467.070000
